In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import os
import importlib
import logging
import matplotlib.pyplot as plt
import corner
from tqdm import tqdm
import matplotlib.pyplot as plt

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
importlib.reload(logging)
logging.basicConfig(level = logging.INFO)

# limit GPU memory
gpus = tf.config.experimental.list_physical_devices('GPU')
try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=20000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
except RuntimeError as e:
# Visible devices must be set before GPUs have been initialized
    print(e)

In [ ]:
training_data = np.load("../../Data/n1000000_0910_all_flat.npz")
data_all = np.column_stack([training_data['ve_dune'][:,:36], training_data['vu_dune'][:,:36], training_data['vebar_dune'][:,:36], training_data['vubar_dune'][:,:36]])

# theta13, theta23, delta
target = np.column_stack([training_data["theta13"]/180*np.pi, training_data["theta23"]/180*np.pi,
                         np.cos(training_data["delta"]/180*np.pi), np.sin(training_data["delta"]/180*np.pi)])

split = 900000
x_train = data_all[:split]
y_train = target[:split]
x_train_poisson = np.random.poisson(x_train)/1000

x_val = data_all[split:]
y_val = target[split:]
x_val_poisson = np.random.poisson(x_val)/1000

In [ ]:
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = tf.keras.Sequential([
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)/10))])
    return prior_model

def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = tf.keras.Sequential([
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ), tfp.layers.MultivariateNormalTriL(n),])
    return posterior_model

In [ ]:
def create_bnn_model(train_size):
    inputs = tf.keras.Input(shape=(len(x_train[0]),), name = 'input')
    features = tf.keras.layers.BatchNormalization()(inputs)
    features = tf.keras.layers.Dense(1024, activation='relu')(features)
    features = tf.keras.layers.Dropout(.5)(features)
    features = tf.keras.layers.Dense(1024, activation='relu')(features)
    features = tf.keras.layers.Dropout(.5)(features)
    features = tf.keras.layers.Dense(1024, activation='relu')(features)

    for units in [16, 16]:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="linear",
        )(features)
    features = tf.keras.layers.Dense(4, activation='linear')(features)
    outputs = features
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
bnn = create_bnn_model(split)
bnn.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.MeanSquaredError()],
)

In [ ]:
bnn.fit(x_train, y_train,
            validation_data=(x_val, y_val),
            batch_size=1024,
            epochs=1000,
            verbose=1,
            shuffle = True
            )

In [ ]:
bnn.save_weights('./bnn/weight_1.h5')

In [ ]:
data = np.load('../../Data/sample_NuFit0911.npz')
data_all = np.column_stack([data['ve_dune'][:,:36], data['vu_dune'][:,:36], data['vebar_dune'][:,:36], data['vubar_dune'][:,:36]])

In [ ]:
result = []
for _ in tqdm(np.arange(10000)):
    result.append(bnn.predict(data_all[:1, :])[0])
result = np.array(result)

In [ ]:
np.savez('./contours/bnn.npz', result)

In [ ]:
angle = np.angle(result[:, 2] + 1j*result[:, 3], deg=True)
angle = np.where(angle > 0 , angle, angle+360)
plt.hist(result[:, 1]*180/np.pi, bins = 100)
plt.show()
plt.scatter(result[:, 1]*180/np.pi, angle)
plt.show()

In [ ]:
corner.hist2d(result[:, 1]*180/np.pi, angle,
                    levels=(0.68,),
                    scale_hist=True,
                    plot_datapoints=False,
                    color='green',
                    labels= ["$\\theta_{23} $($^\circ$)", "$\delta_{cp} $($^\circ$)"],
                    # range=range,
                    plot_contours = True,
                    plot_density = False,
                    fontsize=30,
                    bins = [200, 200],
                    label_kwargs={"fontsize": 30},
                    smooth=True
                   )